In [ ]:
import shutil
import os

import numpy as np

import fiftyone as fo
import fiftyone.zoo as foz
#import fiftyone.core.storage as fos

In [ ]:
if fo.dataset_exists('qs-00'):
    fo.delete_dataset('qs-00')
    
ds = foz.load_zoo_dataset('quickstart',dataset_name='qs-00');
ds = ds.limit(30).clone()
if fo.dataset_exists('qs-00'):
    fo.delete_dataset('qs-00')


ds.name = 'qs-00'
ds.persistent=False

for sidx,s in enumerate(ds):
    fp = s.filepath
    fpdir,fpbase = os.path.split(fp)
    fpbase = os.path.splitext(fpbase)[0]
    if sidx % 2 == 0:
        fpbase += "_montreal"
    fpbase += f'_trial{sidx % 4}'
    
    fpnew = os.path.join(fpdir,fpbase+'.jpg')
    shutil.copyfile(fp,fpnew)
    print(f'moved {fp} -> {fpnew}')
    s.filepath = fpnew
    s.save()
        

In [ ]:
ds.rename_sample_field('ground_truth','labels')
ds.rename_sample_field('predictions','model')

In [ ]:
haz_max = 10
hazard = haz_max*np.random.random(len(ds))
status = ['released','admitted',None]*(len(ds)//3)
fall_hazard = np.random.random(len(ds)).tolist()
fall_hazard[3] = None
fall_hazard[7] = None
ds.set_values('hazard',hazard)
ds.set_values('status',status)
ds.set_values('fall_hazard',fall_hazard)

In [ ]:
# keypts
keypt_classes = ['head','armL','armR','legL','legR']
nclass = len(keypt_classes)
for s in ds:
    keypts = []
    for det in s.labels.detections:
        cls = keypt_classes[np.random.randint(0,nclass)]
        pts = np.random.rand(nclass,2)
        kp = fo.Keypoint(label=cls,points=pts)
        keypts.append(kp)
    s['labels_kpts'] = fo.Keypoints(keypoints=keypts)
    s.save()
    
    keypts = []
    for det in s.model.detections:
        cls = keypt_classes[np.random.randint(0,nclass)]
        pts = np.random.rand(nclass,2)
        kp = fo.Keypoint(label=cls,points=pts)
        keypts.append(kp)
    s['model_kpts'] = fo.Keypoints(keypoints=keypts)
    s.save()
    
        


In [ ]:
ds.name = 'qs-keypts'
ds.persistent=True